In [ ]:
# Import necessary libraries
import os, mne, re
from mne.channels import make_standard_montage
from glob import glob
from mne_bids import BIDSPath, write_raw_bids

# Set the source directory path for the raw data
root_source = 'C:/Users/mfbpe/Desktop/DATA/2023_Eva_Freedom/raw'

# Set the folder for recreate the data in bids format downsampled 
root_bids = 'C:/Users/mfbpe/Desktop/DATA/2023_Eva_Freedom/bids/'

# Get a list of all file paths with .bdf extension in the source directory
all_files_path = sorted(glob(f'{root_source}/*.bdf'), key=len)

# Loop over each file path in the list of all_files_path
for part in all_files_path:
    # Extract the participant number from the file path
    n_part = re.split('Pa|.bdf', part)[1]  # e.g., '001'
    # Read the raw data from the BDF file
    raw = mne.io.read_raw_bdf(part)
    
    bidspath = BIDSPath(subject=n_part, task='constraint', datatype='eeg', root=root_bids)
    
    # Load the data into memory
    raw.load_data()

    events= mne.find_events(raw, shortest_event=1)
    try :
        raw.crop(((events[1,0]/2048))-3, (events[-1,0]/2048)+3)
    except : 
        pass

    events= mne.find_events(raw, shortest_event=1)
    # # Resample the data by downsampling to 1/4th of the original sample rate
    raw, events = raw.resample(sfreq=raw.info['sfreq'] / 4, events= events)
    
    event_id = { 'vizualisation/free':1,'vizualisation/recommended':2,'vizualisation/semi-instructed':3,'vizualisation/instructed':4,
    'action/free':17,'action/recommended':18,'action/semi-instructed':19,'action/instructed':20,
    'outcome/free':33,'outcome/recommended':34,'outcome/semi-instructed':35,'outcome/instructed':36
    }# Define event dictionary

    events = events[np.isin(events[:, 2], list(event_id.values()))]
    raw.add_events(events,replace=True)

    # Set the 32 system BioSemi channel positions on the data
    montage = make_standard_montage('biosemi64')
    new_names = dict(zip(raw.ch_names, montage.ch_names))
    raw.rename_channels(new_names)
    raw.set_montage(montage)

    write_raw_bids(raw, bidspath, events=events, event_id=event_id, format='BrainVision', allow_preload=True,overwrite=True)
